In [205]:
import numpy as np
import pandas as pd
import os
import csv
import time
from PIL import Image
import unittest

import os
import pandas as pd
import json

import time

class Data():
    @staticmethod
    def lidar_array():
        arr = np.random.randint(0,2, size=365)
        return arr
    
    @staticmethod
    def camera_img():
        img = Image.open('/home/wroscoe/mydonkey_old2/sessions/1115am/frame_00001_ttl_0.42665756334488203_agl_0.09659284488019566_mil_0.0.jpg')
        return img
    
    def camera_arr(self):
        img = self.camera_img()
        return np.array(img)
    
    @staticmethod
    def steering_angle():
        return np.random.random() * 2 - 1
    
    @staticmethod
    def throttle():
        return np.random.random() * 1.2 - .2

In [206]:


class Tub():
    """
    A datastore to store sensor data in a key, value format.
    
    Accepts str, int, float, image_array, image, and array data types.
    
    For example:
    
    #Create a tub to store speed values.
    >>> path = '~/mydonkey/test_tub'
    >>> inputs = ['user/speed', 'cam/image']
    >>> types = ['float', 'image']
    >>> t=Tub(path=path, inputs=inputs, types=types)
    
    """
    
    def __init__(self, path, meta=None):
               
        self.path = os.path.expanduser(path)
        self.meta_path = os.path.join(self.path, 'meta.json')
        
        exists = os.path.exists(self.path)
        
        if exists:
            #load log and meta
            with open(self.meta_path, 'r') as f:
                self.meta = json.load(f)
                
            self.current_ix = self.get_last_ix() + 1
                
        elif not exists and meta:
            #create log and save meta
            os.makedirs(self.path)    
            self.meta = meta
            with open(self.meta_path, 'w') as f:
                json.dump(self.meta, f)
            self.current_ix = 0
        else:
            raise AttributeError('The path doesnt exist and you pass meta info.')
        
        self.start_time = time.time()
        
        
    def get_last_ix(self):
        index = self.get_index()
        return max(index)
    
    def get_index(self):
        files = next(os.walk(self.path))[2]
        record_files = [f for f in files if f[:6]=='record']
        print(record_files)
        def get_file_ix(file_name):
            try:
                name = file_name.split('.')[0]
                num = int(name.split('_')[1])
            except:
                num = 0
            return num
        
        nums = [get_file_ix(f) for f in record_files]
        return nums
    
    @property
    def inputs(self):
        return list(self.meta.keys())
    
    @property
    def types(self):
        return list(self.meta.values())

        
    def get_input_type(self, key):
        return self.meta.get(key)
    
    def write_json_record(self, json_data):
        path = self.get_json_record_path(self.current_ix)
        with open(path, 'w') as fp:
            json.dump(json_data, fp)
            
        print(path)
            
    def get_json_record_path(self, ix):
        return os.path.join(self.path, 'record_'+str(ix)+'.json')
    
    def get_json_record(self, ix):
        path = self.get_json_record_path(ix)
        with open(path, 'r') as fp:
            json_data = json.load(fp)
        return json_data
        
    def put_record(self, data):
        """
        Save values like images that can't be saved in the csv log and
        return a record with references to the saved values that can
        be saved in a csv.
        """
        
        json_data = {}
        
        line = []
        
        for key, val in data.items():
            typ = self.get_input_type(key)
            
            if typ in ['str', 'float', 'int']:
                json_data[key] = val       

            elif typ is 'image':
                path = self.make_file_path(key)
                val.save(path)
                json_data[key]=path
                
            elif typ == 'image_array':
                path = self.make_file_path(key, ext='.png')
                img = Image.fromarray(np.uint8(val))
                img.save(path)
                json_data[key]=path

            else:
                msg = 'Tub does not know what to do with this type {}'.format(typ)
                raise TypeError(msg)
        
        #write csv line
        self.write_json_record(json_data)
        self.current_ix += 1
    
    def get_record(self, ix):
        
        json_data = self.get_json_record(ix)
        data={}
        for key, val in json_data.items():
            typ = self.get_input_type(key)
            
            #load objects that were saved as separate files
            if typ == 'image':
                val = Image.open(val)
            elif typ == 'image_array':
                img = Image.open(val)
                val = np.array(img)
            
            data[key] = val
            
        return data
            
    @staticmethod
    def clean_file_name(name):
        name = name.replace('/', '-')
        return name
    
    def make_file_path(self, key, ext='.png'):
        name = '_'.join([str(self.current_ix), key, ext])
        name = self.clean_file_name(name)
        file_path = os.path.join(self.path, name)
        return file_path
        
    def delete(self):
        """ Delete the folder and files for this tub. """
        import shutil
        shutil.rmtree(self.path)
        
    def record_gen(self, index):
        while True:
            for i in index:
                record = self.get_record(i)
                yield record
                
    def batch_gen(self, keys, index, batch_size=32):
        record_gen = self.record_gen(index)
        while True:
            record_list = []
            for _ in range(batch_size):
                record_list.append(next(record_gen))
            
            batch_arrays = {} 
            for i, k in enumerate(keys):
                arr = np.array([r[k] for r in record_list])
                #if len(arr.shape) == 1:
                #    arr = arr.reshape(arr.shape + (1,))
                batch_arrays[k] = arr
                
            yield batch_arrays
    

In [195]:
t = Tub('/home/wroscoe/donk2/data/tub_test2', 
        meta={'x':'float', 'y':'float', 'z':'image_array'})

d = Data()

['record_1.json', 'record_6.json', 'record_4.json', 'record_3.json', 'record_5.json', 'record_2.json']


In [196]:
index = t.get_index()


['record_1.json', 'record_6.json', 'record_4.json', 'record_3.json', 'record_5.json', 'record_2.json']


In [197]:
record_gen = t.record_gen(index=index)

In [198]:
batch_gen = t.batch_gen(keys=['x', 'z', 'y'], index=index)

In [184]:
arr = next(batch_gen)

In [185]:
arr['x'].shape

(32,)

In [190]:
import keras
from keras.layers import Input, Dense, merge
from keras.models import Model
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, SimpleRNN, Reshape, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.regularizers import l2

img_in = Input(shape=(120, 160,3), name='img_in')
x = img_in
x = Convolution2D(24, (5,5), strides=(2,2), activation='relu')(x)
x = Convolution2D(32, (5,5), strides=(2,2), activation='relu')(x)
x = Convolution2D(64, (5,5), strides=(2,2), activation='relu')(x)
x = Convolution2D(64, (3,3), strides=(2,2), activation='relu')(x)
x = Convolution2D(64, (3,3), strides=(1,1), activation='relu')(x)

x = Flatten(name='flattened')(x)
x = Dense(100, activation='relu')(x)
x = Dropout(.1)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(.1)(x)
#categorical output of the angle
angle_out = Dense(1, activation='relu', name='angle_out')(x)

#continous output of throttle
throttle_out = Dense(1, activation='relu', name='throttle_out')(x)

model = Model(inputs=[img_in], outputs=[angle_out, throttle_out])


model.compile(optimizer='rmsprop',
              loss={'angle_out': 'mean_absolute_error', 
                    'throttle_out': 'mean_absolute_error'},
              loss_weights={'angle_out': 0.9, 'throttle_out': .1})

In [208]:
model_path = '/home/wroscoe/donk2/models/test_model'

#checkpoint to save model after each epoch
save_best = keras.callbacks.ModelCheckpoint(model_path, monitor='loss', verbose=1, 
                                      save_best_only=True, mode='min')

#stop training if the validation error stops improving.
early_stop = keras.callbacks.EarlyStopping(monitor='loss', min_delta=.0005, patience=4, 
                                     verbose=1, mode='auto')

callbacks_list = [save_best, early_stop]
steps =10
epochs=10
model.fit(x = arr['z'], y=[arr['x'], arr['y']], epochs=10)


Epoch 1/10
32/32 [==============================] - 0s - loss: 541131.3750 - angle_out_loss: 600639.8750 - throttle_out_loss: 5555.0000
Epoch 2/10
32/32 [==============================] - 0s - loss: 606924.9375 - angle_out_loss: 673743.8125 - throttle_out_loss: 5555.0000
Epoch 3/10
32/32 [==============================] - 0s - loss: 592784.6875 - angle_out_loss: 657673.7500 - throttle_out_loss: 8783.3340
Epoch 4/10
32/32 [==============================] - 0s - loss: 710084.2500 - angle_out_loss: 788365.3125 - throttle_out_loss: 5555.0000
Epoch 5/10
32/32 [==============================] - 0s - loss: 497573.4062 - angle_out_loss: 550685.0000 - throttle_out_loss: 19569.0312
Epoch 6/10
32/32 [==============================] - 0s - loss: 568609.3750 - angle_out_loss: 631171.0000 - throttle_out_loss: 5555.0000
Epoch 7/10
32/32 [==============================] - 0s - loss: 681123.5625 - angle_out_loss: 756186.7500 - throttle_out_loss: 5555.0000
Epoch 8/10
32/32 [=============================

In [209]:
def train_gen(gen, X_keys, y_keys):
    while True:
        batch = next(gen)
        X = [batch[k] for k in X_keys]
        y = [batch[k] for k in y_keys]
        yield X, y
        
    

In [210]:
tg = train_gen(batch_gen, ['z'], ['x', 'y'])

In [211]:
hist = model.fit_generator(
                        tg, 
                        steps_per_epoch=steps, 
                        nb_epoch=epochs, 
                        verbose=1, 
                        callbacks=callbacks_list, 
                        nb_val_samples=steps*.2)

/home/wroscoe/envs/donkey/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=10, callbacks=[<keras.ca..., verbose=1, epochs=10, validation_steps=2.0)`
  import sys


Epoch 1/10
10/10 [==============================] - 4s - loss: 604441.8250 - angle_out_loss: 670940.6031 - throttle_out_loss: 5952.9059     
Epoch 2/10
 8/10 [=======================>......] - ETA: 0s - loss: 600664.8164 - angle_out_loss: 666767.9062 - throttle_out_loss: 5737.1855

KeyboardInterrupt: 

In [212]:
from PIL import Image


In [ ]:
Image.open()